In [39]:
import pandas as pd
import numpy as np
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm_notebook as tqdm
from collections import defaultdict

In [2]:
path = Path('/kaggle/input/data-science-bowl-2019/')

## Read in data

In [3]:
print('Reading in train.csv')
train_data = pd.read_csv(path/'train.csv')
print('Reading in train_labels.csv')
train_labels = pd.read_csv(path/'train_labels.csv')
print('Reading in test.csv')
test_data = pd.read_csv(path/'test.csv')
print('Reading in specs.csv')
specs = pd.read_csv(path/'specs.csv')

Reading in train.csv
Reading in train_labels.csv
Reading in test.csv
Reading in specs.csv


## Reduce the size of the data frames

In [4]:
## Function to reduce the DF size
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [5]:
train_data = reduce_mem_usage(train_data)
train_labels = reduce_mem_usage(train_labels)
test_data = reduce_mem_usage(test_data)
specs = reduce_mem_usage(specs)

Mem. usage decreased to 778.73 Mb (18.2% reduction)
Mem. usage decreased to  0.49 Mb (48.2% reduction)
Mem. usage decreased to 79.40 Mb (18.2% reduction)
Mem. usage decreased to  0.01 Mb (0.0% reduction)


In [6]:
# helper function to decompose dates into more relevant parts
def explode_date(col):
    dayofyear = col.dt.dayofyear
    weekofyear = col.dt.weekofyear
    weekday = col.dt.weekday
    month = col.dt.month
    year = col.dt.year
    hour = col.dt.hour
    quarter = col.dt.quarter
    return pd.DataFrame({'dayofyear':dayofyear,
                         'weekofyear':weekofyear,
                         'weekday':weekday,
                         'quarter':quarter,
                         'month':month,
                         'year':year,
                         'hour':hour})

Main funtion that aggregates the sequence data

In [113]:
# reduces data with respect to each installation id
def reduce_data_per_user(user_dataframe):
    event_counts = {'total_events':0,'clips_count':0,'assessment_count':0,'activity_count':0,'game_count':0,
                    'correct_events_count':0,'incorrect_events_count':0}
    specific_event_counts = dict(zip(specs.event_id,[0 for _ in range(specs.shape[0])]))
    df_features = []
    for idx,session in user_dataframe.groupby('game_session'):
        if session.type.iloc[0]=='Assessment':
            assessment_info = {'assess_world':session.world.iloc[0],
                               'assess_title':session.title.iloc[0],
                               'game_session':session.game_session.iloc[0],
                               'installation_id':session.installation_id.iloc[0]
                              }
            df_features.append({**event_counts,**assessment_info,**specific_event_counts})
            event_counts['assessment_count'] += (session.type == 'Assessment').sum()
        else:
            event_counts['total_events'] += session.event_id.count()
            event_counts['clips_count'] += (session.type == 'Clip').sum()
            event_counts['activity_count'] += (session.type == 'Activity').sum()
            event_counts['game_count'] += (session.type == 'Game').sum()
            event_counts['correct_events_count'] += session.event_id.str.contains('Correct').sum()
            event_counts['incorrect_events_count'] += session.event_id.str.contains('Incorrect').sum()
            specific_event_counts.update(dict(session.event_id.value_counts())) 
    return df_features
#pd.DataFrame(reduce_data_per_user(train_data.loc[train_data.installation_id == unique_install_ids[0]]))

In [110]:
def reduce_data(install_id,data):
    rows = []
    for idx,df in tqdm(data.groupby('installation_id')):
        rows.extend(reduce_data_per_user(df))
    return pd.DataFrame(rows)

train_reduced = reduce_data(train_labels.installation_id.unique(),train_data)
test_reduced = reduce_data(test_data.installation_id.unique(),test_data)

In [111]:
test_reduced.head()

,total_events,clips_count,assessment_count,activity_count,game_count,correct_events_count,incorrect_events_count,assess_world,assess_title,game_session,...,7d5c30a2,d2278a3b,99ea62f3,9c5ef70c,7040c096,29f54413,06372577,2a444e03,9e6b7fb5,d3640339
0,145,5,0,61,79,0,0,MAGMAPEAK,Cauldron Filler (Assessment),348d7f09f96af313,...,0,0,0,0,0,0,0,0,0,0
1,528,9,1,440,79,0,0,CRYSTALCAVES,Cart Balancer (Assessment),8b38fc0d2fd315dc,...,0,0,0,0,0,0,0,0,0,0
2,1,1,0,0,0,0,0,MAGMAPEAK,Cauldron Filler (Assessment),009c890ce6c4f3e3,...,0,0,0,0,0,0,0,0,0,0
3,304,4,36,50,250,0,0,CRYSTALCAVES,Cart Balancer (Assessment),1fef5d54cb4b775a,...,0,1,0,1,1,0,0,17,0,0
4,304,4,37,50,250,0,0,TREETOPCITY,Mushroom Sorter (Assessment),31423dbcd717919e,...,0,1,0,1,1,0,0,17,0,0


# Modeling

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import cohen_kappa_score,confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split

In [ ]:
train,val = train_test_split(train_data_agg,test_size=.2)
train_x = train.iloc[:,:-1]
train_y = train.iloc[:,-1]
val_x = val.iloc[:,:-1]
val_y = val.iloc[:,-1]

In [ ]:
preprocessor = ColumnTransformer([('drop','drop',[0]),
                                  ('onehot',OneHotEncoder(),['assessment_title','assessment_world'])],remainder='passthrough')

In [ ]:
pipe = Pipeline([('col_transform',preprocessor),
                 ('cls',RandomForestClassifier())])

In [ ]:
pipe_fit = pipe.fit(train_x,train_y)

In [ ]:
params = {'cls__n_estimators':range(1,300,10),
          'cls__max_depth':range(1,15,2),
          'cls__min_samples_leaf':range(10,50,2)}

In [ ]:
cv = RandomizedSearchCV(pipe,params,n_jobs=10,n_iter=10)
cv.fit(all_x,all_y)

In [ ]:
preds = cv.predict(val_x)
quadratic_kappa(val_y,preds)

In [ ]:
test_agg = aggregate(test_data,'installation_id')
test_agg.head()

In [ ]:
def prepare_submission():
    test_agg = aggregate(test_data,'installation_id')
    test_predictions = cv.predict(test_agg)
    submission_df = pd.DataFrame({'installation_id':test_agg.installation_id.to_numpy(),
                                  'accuracy_group':test_predictions})
    return submission_df
test = prepare_submission()

In [ ]:
test.to_csv('submission.csv',index=False)

In [ ]:
test.to_csv('submission/submission.csv',index=False)